In [1]:
# TO IMPORT THE CODE WRITTEN IN SRC
import sys
sys.path.append('..')

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file, loads openai and tavily keys
openai.api_key = os.environ['OPENAI_API_KEY']
from langchain_openai import ChatOpenAI


## building faiss retriever from langchain url

In [4]:
from src.retrieval_tools.retrieval_utils import *

In [5]:
url = "https://docs.smith.langchain.com/overview"
docs = docs_loaded_from_web(url)
retriever = build_whole_text_recursive_faiss_retriever(docs)


## building tools

### tavily retriever

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

### langchain retriever tool

In [7]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [8]:
tools = [retriever_tool, search]

## Building the agent and execute it

In [9]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Testing different scenarios to see which tool is used

In [10]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in several ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.

2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.

3. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine which parts of the software have been exercised during testing. It can provide insights into the effectiveness of the test suite and identify areas that need additional testing.

4. Test Prioritization: LangSmith can prioritize test cases based on the

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in several ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on the specifications of the software being tested. It uses advanced techniques such as symbolic execution and constraint solving to generate test inputs that cover different execution paths and edge cases.\n\n2. Test Oracles: LangSmith can help in defining test oracles, which are the expected outputs or behaviors of the software under test. It can analyze the specifications and requirements of the software to automatically generate assertions or checks that can be used as test oracles.\n\n3. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine which parts of the software have been exercised during testing. It can provide insights into the effectiveness of the test suite and identify areas that need additional testing.\n\n4. Test Prioritization: LangSmith can prio

In [11]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://weatherspark.com/h/m/557/2024/1/Historical-Weather-in-January-2024-in-San-Francisco-California-United-States', 'content': 'January 2024 Weather History in San Francisco California, United States  Daily Precipitation in January 2024 in San Francisco Observed Weather in January 2024 in San Francisco  San Francisco Temperature History January 2024 Hourly Temperature in January 2024 in San Francisco  Hours of Daylight and Twilight in January 2024 in San FranciscoLatest Report — 11:56 AM Fri, Jan 12, 2024 7 min ago UTC 19:56 Call Sign KSFO Temp. 52.0 °F cold Precipitation No Report Wind 6.9 mph light breeze Wind Dir. 80 deg, E Cloud Cover Mostly Cloudy 18,000 ft Mostly Clear 4,000 ft Partly Cloudy 15,000 ft Raw: KSFO 121956Z 08006KT 10SM FEW040 SCT150 BKN180 11/07 A3028 RMK AO2 SLP254 T01110067'}, {'url': 'https://www.whereandwhen.net/when/north-am

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is 52.0 °F with a light breeze of 6.9 mph coming from the east. The sky is mostly cloudy with some clear and partly cloudy areas.'}

In [12]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.

2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.

3. Essay Editing: LangSmith can assist with editing and proofreading your application essays, ensuring they are well-written, coherent, and free of grammatical errors. This can help you present your ideas clearly and professionally.

4. Interview Preparation: LangSmith can conduct mock interviews and provide feedback on your performance. This can help you practice and improve your interview skills, incre

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.\n\n2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.\n\n3. Essay Editing: LangSmith can assist with editing and proofreading your application essays, ensuring they are well-written, coherent, and free of grammatical errors. This can help you present your ideas clearly and professionally.\n\n4. Interview Preparation: LangSmith can conduct mo